<a href="https://colab.research.google.com/github/samyrsp86/alura-obrai/blob/main/ObrAI_Conversas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instala no notebook as ferramentas e funções necessárias para usar o SDK da IA Generativa do Google

%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Captura a imagem da URL informada pelo usurário
from PIL import Image
import requests
import io
from IPython.display import Image as DisplayImage # Importar com um nome diferente para evitar conflitos

def exibir_imagem_do_colab(url):

     try:
         # Envia uma requisição GET para obter a imagem
         resposta = requests.get(url)
         resposta.raise_for_status()  # Lança uma exceção para códigos de status ruins


         # Abre a imagem a partir do conteúdo da resposta
         imagem_bytes = io.BytesIO(resposta.content)
         imagem = Image.open(imagem_bytes)


         # Exibe a imagem no Colab
         DisplayImage(imagem) # Utilizo o nome 'DisplayImage' aqui
         return imagem # Retorna o objeto Image para uso posterior, se necessário


     except requests.exceptions.RequestException as e:
         print(f"Erro ao baixar a imagem da URL {url}: {e}")
         return None
     except Exception as e:
         print(f"Erro ao abrir ou exibir a imagem: {e}")
         return None

In [ ]:
# Fornece ao Gemini às instruções a serem consideradas na interpretação da imagem fornecida

from google.genai import types

def interpreta_nota_fiscal(imagem = None, resposta_inicial = None, consideracao_usuario = None):

  chat_config = types.GenerateContentConfig(system_instruction = (
                          """Você receberá até três parâmetros, a url uma nota fiscal no formato de imagem ou pdf e possivelmente sua própria resposta,
                          acompanhada uma consideração do usuário sobre interpretação da Nota Fiscal.

                          Considerando os parâmetros recebidos, você deverá buscar neste arquivo as seguintes informações:
                          Nome do fornecedor, Data da emissão, Número da nota fiscal, Descrição do produto ou serviço (restrito a um resumo de 50 caracteres, quando aplicável),
                          Quantidade, Valor unitário.
                          Entenda que há mais de um padrão de nota fiscal. A nota carioca de serviço, por exemplo, possui um padrão diferente e a informação do fornecedor
                          poderá ser encontrada no campo "Prestador de Serviços", já a informação da Descrição do produto/serviço pode ser encontrada no campo "Discriminação dos Serviços".
                          Caso seja um serviço deverá ser considerada apenas uma unidade.

                          Você deverá confirmar com o usuário se os dados foram interpretados de forma correta.
                          Caso não tenham, você deve solicitar auxílio do usuário nesta interpretação até que esteja correta.
                          A sua resposta final deverá conter única e exclusivamente os itens interpretação, conforme modelo apresentado abaixo.

                          Caso a nota fiscal possua mais de um item, você deve repetir os dados comuns para cada item, respeitando o template apresentado a seguir:

                          ##### Item 1 #####

                          Nome do fornecedor: FERRAGENS BUARQUE MACEDO EIRELLI
                          Data de emissão: 16/01/2025
                          Número da nota fiscal: 365045
                          Descrição do produto/serviço: CANT.ALUM.TL C 3MT 1 2
                          Quantidade: 2
                          Valor unitário: 12,950000

                          ##### Item 2 #####

                          Nome do fornecedor: FERRAGENS BUARQUE MACEDO EIRELLI
                          Data de emissão: 16/01/2025
                          Número da nota fiscal: 365045
                          Descrição do produto/serviço: TRI PARAF.PHI.C CHATA 2,5X25 11 4X4
                          Quantidade: 10
                          Valor unitário: 12,950000

                          Se você não conseguir ler exatamente as informações da nota fiscal fornecida, deverá dizer: 'Não foi possível ler o arquivo!'
                          Também deverá apresentar uma justificativa para não ter conseguido ler"""))

  chat = client.chats.create(model = MODEL_ID, config= chat_config)

  mensagem = imagem
  if consideracao_usuario:
    mensagem = f"A sua resposta foi: {resposta_inicial}\n\nConsideração do usuário: {consideracao_usuario}"

  resposta = chat.send_message(mensagem)

  return resposta.text

In [ ]:
# Instala Framework ADK de agentes do Google
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types  # Para criar conteúdos (Content e Part)
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
################################################
# --- Agente 1: Categorizador de Gastos 1 --- #
################################################
def agente_categorizador_1(resultado_interpretacao, consideracao_usuario_categorizacao = None):
    categorizador_1 = Agent(
        name="agente_categorizador_1",
        model="gemini-2.0-flash",
        # Inserir as instruções do 1º Agente Categorizador
        instruction="""
        Você é um categorizador de gastos. Você receberá como parâmetros os gastos contantes de uma nota fiscal e, possivelmente, as considerações do usuário sobre a sua tentativa de categorização.

        Poderá ter um ou mais gastos nesta nota.
        Sua função será adicionar o campo "Tipo de Gasto 1" para cada um dos itens (ou item) que lhe for informado.
        Para isso você deverá classificar os itens seguindo, necessariamente, alguma das categorias a seguir:
        Alvenaria
        Hidraulica
        Elétrica
        Marcenaria
        Móveis
        Pintura
        Eletrodomésticos
        Caso não consiga classificar em nenhum destes itens, deverá colocar o rótulo Outras Despesas.

        Você receberá os itens conforme a lista a seguir:

        Item 1
        Nome do fornecedor: FERRAGENS BUARQUE MACEDO EIRELLI
        Data de emissão: 16/01/2025
        Número da nota fiscal: 365045
        Descrição do produto/serviço: TIJOLO 29X39
        Quantidade: 200
        Valor unitário: 12,950000

        Item 2
        Nome do fornecedor: FERRAGENS BUARQUE MACEDO EIRELLI
        Data de emissão: 16/01/2025
        Número da nota fiscal: 365045
        Descrição do produto/serviço: TRI PARAF.PHI.C CHATA 2,5X25 11 4X4
        Quantidade: 10
        Valor unitário: 0,500000

        Seguindo este exemplo o seu output deverá ser:

        Item 1
        Nome do fornecedor: FERRAGENS BUARQUE MACEDO EIRELLI
        Data de emissão: 16/01/2025
        Número da nota fiscal: 365045
        Descrição do produto/serviço: TIJOLO 29X39
        Quantidade: 200
        Valor unitário: 12,950000
        Tipo de Gasto 1: Alvenaria

        Item 2
        Nome do fornecedor: FERRAGENS BUARQUE MACEDO EIRELLI
        Data de emissão: 16/01/2025
        Número da nota fiscal: 365045
        Descrição do produto/serviço: TRI PARAF.PHI.C CHATA 2,5X25 11 4X4
        Quantidade: 10
        Valor unitário: 0,500000
        Tipo de Gasto 1: Outras Despesas

        Você deverá solicitar a confirmação do usuário de que os dados foram classificados de forma correta.
        Caso não tenham, você deve solicitar auxílio do usuário nesta primeira classificação até que a classificação esteja correta.
        A sua resposta final deverá conter única e exclusivamente os itens da primeira classificação, conforme modelo apresentado acima.

        """,
        description="Agente que categoriza gastos em notas fiscais",
    )



    if consideracao_usuario_categorizacao:
      entrada_do_agente_categorizador_1 = f"Os dados da Nota Fiscal: {resultado_interpretacao}\n\nConsideração do usuário: {consideracao_usuario_categorizacao}"
    else:
      entrada_do_agente_categorizador_1 = f"Os dados da Nota Fiscal:{resultado_interpretacao}\n"


    # Executa o agente
    dados_categorizados_1 = call_agent(categorizador_1, entrada_do_agente_categorizador_1)
    return dados_categorizados_1

In [ ]:
######################################
# --- Agente 2: Categorizador 2 --- #
######################################
def agente_categorizador_2(dados_categorizados_1, consideracao_usuario_categorizacao_2 = None):
    categorizador_2 = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        # Inserir as instruções do 2º Agente Categorizador
        instruction="""
            Você é um agente categorizador. Sua função é analizar as informações oriundas de uma nota fiscal, que foram previamente
            categorizadas no campo "Tipo de Gasto 1" e inserir o campo "Tipo de Gasto 2" que deverá receber um dos dois valores: "Não Incorporável ao IR" e "Incorporável ao IR".
            Para realizar esta classificação, você deverá analisar os campos Descrição do produto/serviço e "Tipo de Gasto 1" com a finalidade de entender se aquele pode ou não ser
            um gasto que pode ser incorporado ao valor venal do imóvel no momento da declaração do imposto de renda pessoa física.
            É algo parecido com o que acontece na contabilidade gerencial, em que custos e despesas (OPEX) não podem ser incorporadas ao imobilizado e investimentos podem.
            Em geral, itens que ficam soltos como eletrodomésticos e móveis não podem ser incorporados ao IR. Mas a marcenaria, por exemplo, pode ser incorporado.

            Você receberá os itens conforme a lista a seguir:

            Item 1
            Nome do fornecedor: FERRAGENS BUARQUE MACEDO EIRELLI
            Data de emissão: 16/01/2025
            Número da nota fiscal: 365045
            Descrição do produto/serviço: TIJOLO 29X39
            Quantidade: 200
            Valor unitário: 12,950000
            Tipo de Gasto 1: Alvenaria

            Item 2
            Nome do fornecedor: FERRAGENS BUARQUE MACEDO EIRELLI
            Data de emissão: 16/01/2025
            Número da nota fiscal: 365045
            Descrição do produto/serviço: TRI PARAF.PHI.C CHATA 2,5X25 11 4X4
            Quantidade: 10
            Valor unitário: 0,500000
            Tipo de Gasto 1: Outras Despesas

            Seguindo este exemplo o seu output deverá ser:

            Item 1
            Nome do fornecedor: FERRAGENS BUARQUE MACEDO EIRELLI
            Data de emissão: 16/01/2025
            Número da nota fiscal: 365045
            Descrição do produto/serviço: TIJOLO 29X39
            Quantidade: 200
            Valor unitário: 12,950000
            Tipo de Gasto 1: Alvenaria
            Tipo de Gasto 2: Incorporável ao IR

            Item 2
            Nome do fornecedor: FERRAGENS BUARQUE MACEDO EIRELLI
            Data de emissão: 16/01/2025
            Número da nota fiscal: 365045
            Descrição do produto/serviço: TRI PARAF.PHI.C CHATA 2,5X25 11 4X4
            Quantidade: 10
            Valor unitário: 0,500000
            Tipo de Gasto 1: Outras Despesas
            Tipo de Gasto 2: Não Incorporável ao IR

            Você deverá solicitar a confirmação do usuário de que os dados foram classificados de forma correta.
            Caso não tenham, você deve solicitar auxílio do usuário nesta segunda classificação.

            """,
        description="Agente que categoriza gastos em notas fiscais para fins de Imposto de Renda"
    )

    if consideracao_usuario_categorizacao_2:
      entrada_do_agente_categorizador_2 = f"Os dados da Nota Fiscal: {dados_categorizados_1}\n\nConsideração do usuário: {consideracao_usuario_categorizacao_2}"

    else:
      entrada_do_agente_categorizador_2 = f"Itens constantes da Nota fiscal a ser analisada:{dados_categorizados_1}\n"


    # Executa o agente
    dados_categorizados_2 = call_agent(categorizador_2, entrada_do_agente_categorizador_2)
    return dados_categorizados_2

In [ ]:
##########################################
# --- Agente 3: Gerador de JSon --- #
##########################################
def agente_gerador_json(dados_categorizados_2):
    gerador_json = Agent(
        name="agente_gerador_json",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Gerador de Json
        instruction="""
            Você é um gerador de json. Você receberá os dados classificados dos gastos de uma nota fiscal e deverá gerar o conteúdo de um arquivo json,
            que será incorporado em um banco de dados. Seu output será apenas o arquivo json, que deverá sempre ter a mesma estrutura, conforme apresentado a seguir:
            {
                "Nome do fornecedor": "FERRAGENS BUARQUE MACEDO EIRELLI",
                "Data de emissão": "06/12/2024",
                "Número da nota fiscal": "361735",
                "Descrição do produto/serviço": "M COTOV.90 ROSC. 3 4",
                "Quantidade": 5,
                "Valor unitário": 14.700000,
                "Tipo de Gasto 1": "Hidraulica",
                "Tipo de Gasto 2": "Incorporável ao IR"
              }

            """,
        description="Agente gerador de json."
    )
    entrada_gerador_json = f"Gerar json: {dados_categorizados_2}"
    # Executa o agente
    arquivo_json = call_agent(gerador_json, entrada_gerador_json)
    return arquivo_json

In [ ]:
print("🪚🛠️🪛🧰 Iniciando o Sistema ObrAÍ 🪚🛠️🪛🧰")

# --- Obter a URL da imagem da Nota Fiscal  ---

url_da_imagem = input("Por favor, insira a URL da Nota Fiscal que você deseja interpretar e confira o resultado ao final: ")

# Exibe a imagem
imagem = exibir_imagem_do_colab(url_da_imagem)

# Verifica se a imagem foi aberta com sucesso e prossegue se sim
if imagem:
    print("Imagem exibida com sucesso no Colab.")
    # Obtém as dimensões da Imagem para verificar se foi devidamente carregada
    largura, altura = imagem.size
    print(f"A imagem tem largura {largura} e altura {altura}.")
    # Inicia a função de interpretação da Nota Fiscal
    resposta_interpretacao = interpreta_nota_fiscal(imagem)
    print(resposta_interpretacao)

# Questiona o usuário se o Agente interpretou corretamente a Nota Fiscal e entra no Loop para que a correção seja feita
    resultado_interpretacao = input("Interpretei corretamente as Notas Fiscais? Caso negativo, apresente a justificativa, por favor ")

    while resultado_interpretacao.lower().strip() not in "sim":
      resposta_interpretacao = interpreta_nota_fiscal(resposta_inicial = resposta_interpretacao, consideracao_usuario = resultado_interpretacao)
      print(resposta_interpretacao)
      resultado_interpretacao = input("E agora, interpretei corretamente as Notas Fiscais? Caso negativo, apresente a justificativa, por favor ")

else:
    print("Não foi possível exibir a imagem.")


In [ ]:
# Lógica do sistema de agentes

# Carrega os dados já processados da Nota Fiscal para que o Gemini faça a primeira rotina de classificação
dados_categorizados_1 = agente_categorizador_1(resultado_interpretacao=resposta_interpretacao)
print("\n --- Resultado do Agente Categorizador 1 --- \n")
print(dados_categorizados_1)

# Questiona o usuário se o Agente categorizou conforme a natureza da despesa corretamente a Nota Fiscal e entra no Loop para que a correção seja feita
resultado_categorizacao_1 = input("Categorizei corretamente as Notas Fiscais? Caso negativo, apresente a justificativa, por favor ")

while resultado_categorizacao_1.lower().strip() not in "sim":
  dados_categorizados_1 = agente_categorizador_1(resultado_interpretacao = resposta_interpretacao, consideracao_usuario_categorizacao = resultado_categorizacao_1)
  print("\n --- Resultado do Agente Categorizador 1 --- \n")
  print(dados_categorizados_1)
  resultado_categorizacao_1 = input("E agora, interpretei corretamente as Notas Fiscais? Caso negativo, apresente a justificativa, por favor ")



In [ ]:
# Carrega os dados já processador e classificados para que o Gemini faça a segunda rotina de classificação
dados_categorizados_2 = agente_categorizador_2(dados_categorizados_1 = dados_categorizados_1)
print("\n --- Resultado do Agente Categorizador 2 --- \n")
print(dados_categorizados_2)

# Questiona o usuário se o Agente categorizou conforme a possibilidade de incorporação ao valor do imóvel da despesa corretamente a Nota Fiscal e entra no Loop para que a correção seja feita
resultado_categorizacao_2 = input("Categorizei corretamente as Notas Fiscais? Caso negativo, apresente a justificativa, por favor ")

while resultado_categorizacao_2.lower().strip() not in "sim":
  dados_categorizados_2 = agente_categorizador_2(dados_categorizados_1 = dados_categorizados_1, consideracao_usuario_categorizacao_2 = resultado_categorizacao_2)
  print("\n --- Resultado do Agente Categorizador 2 --- \n")
  print(dados_categorizados_2)
  resultado_categorizacao_2 = input("E agora, interpretei corretamente as Notas Fiscais? Caso negativo, apresente a justificativa, por favor ")

In [ ]:
# Carrega os dados totalmente classificados para que se gere o arquivo Json com os dados a serem, futuramente, incorporados ao Banco de Dados
arquivo_json = agente_gerador_json(dados_categorizados_2=dados_categorizados_2)

print("\n --- Resultado do Agente Gerador de JSON --- \n")
print(arquivo_json)
print("----------------------------------------------------")